In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# YOLO

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.0/883.0 kB 12.8 MB/s eta 0:00:00


In [ ]:
import os
import cv2
import requests
from osgeo import gdal
import geopandas as gpd
from shapely.geometry import Point, box
from tqdm import tqdm
from ultralytics import YOLO
from io import BytesIO

# Load the trained YOLO model
checkpoint_path = '/content/drive/MyDrive/aerial_image_recognition/img/train/Tokyo/yolov8_tokyo_checkpoint.pt'
model = YOLO(checkpoint_path)
print("Model loaded from checkpoint successfully!")

# Load the 'ramki.shp' layer using Geopandas
ramki_shp_path = '/content/drive/MyDrive/aerial_image_recognition/gis/shp/ramki.shp'
ramki_gdf = gpd.read_file(ramki_shp_path)

# Select the specific area named 'srodmiescie' (adjust this filter for your case)
srodmiescie_gdf = ramki_gdf[ramki_gdf['name'] == 'srodmiescie']

# Get the bounding box of 'srodmiescie'
minx, miny, maxx, maxy = srodmiescie_gdf.total_bounds
print(f"Bounding box of srodmiescie: {minx}, {miny}, {maxx}, {maxy}")

# Define WMS parameters
wms_url = "https://mapy.geoportal.gov.pl/wss/service/PZGIK/ORTO/WMS/StandardResolution"
wms_params = {
    'service': 'WMS',
    'version': '1.3.0',
    'request': 'GetMap',
    'layers': 'ORTO',
    'styles': '',
    'crs': 'EPSG:3857',  # Match to your projected CRS (Web Mercator)
    'bbox': f'{minx},{miny},{maxx},{maxy}',  # Update for each tile
    'width': 1200,  # Set the image size matching your model
    'height': 1200,
    'format': 'image/tiff'
}

# Define an empty GeoDataFrame for storing car/truck detections
car_centroids_gdf = gpd.GeoDataFrame(columns=['geometry', 'class', 'confidence'])

# Loop through and fetch tiles from the WMS server
stride = 600  # Adjust stride for partial overlap
tile_size = 1200

# Split the area into smaller tiles based on stride and tile size
x_coords = list(range(int(minx), int(maxx), stride))
y_coords = list(range(int(miny), int(maxy), stride))

total_tiles = len(x_coords) * len(y_coords)

with tqdm(total=total_tiles, desc="Processing Tiles") as pbar:
    for x in x_coords:
        for y in y_coords:
            # Adjust bounding box for each tile
            tile_bbox = f'{x},{y},{x+tile_size},{y+tile_size}'
            wms_params['bbox'] = tile_bbox

            # Request the tile from WMS
            response = requests.get(wms_url, params=wms_params)

            if response.status_code == 200:
                # Load the image into OpenCV
                img_data = BytesIO(response.content)
                img = gdal.Open(img_data).ReadAsArray()
                img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

                # Run YOLO inference on the tile
                results = model(img)

                # Extract bounding boxes and append to GeoDataFrame
                for box in results[0].boxes:
                    if box.cls in [0, 1] and box.conf > 0.4:  # Only consider cars/trucks
                        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                        centroid_x = (x1 + x2) / 2 + x  # Adjust based on the tile's position
                        centroid_y = (y1 + y2) / 2 + y

                        # Append the result to the GeoDataFrame
                        car_centroids_gdf = car_centroids_gdf.append({
                            'geometry': Point(centroid_x, centroid_y),
                            'class': 'Car' if box.cls == 0 else 'Truck',
                            'confidence': box.conf.item()
                        }, ignore_index=True)

            pbar.update(1)

# Set CRS to match the WMS (EPSG:3857 for Web Mercator)
car_centroids_gdf.set_crs(epsg=3857, inplace=True)

# Save the results to a GeoJSON file
output_geojson_path = '/content/drive/MyDrive/aerial_image_recognition/gis/car_centroids_2.geojson'
car_centroids_gdf.to_file(output_geojson_path, driver='GeoJSON')

print(f"Car and truck centroids saved to {output_geojson_path}")


Model loaded from checkpoint successfully!
Bounding box of srodmiescie: 20.981389979899493, 52.22652066381057, 21.029888441054208, 52.25128290846448


Processing Tiles: 0it [00:00, ?it/s]

Car and truck centroids saved to /content/drive/MyDrive/aerial_image_recognition/gis/car_centroids_2.geojson


# ONNX

In [3]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.7 MB/s eta 0:00:00


In [16]:
import onnxruntime as ort
import cv2
import os
import geopandas as gpd
from shapely.geometry import Point
from tqdm import tqdm
from osgeo import gdal
import numpy as np

# Load the ONNX model
onnx_model_path = '/content/drive/MyDrive/aerial_image_recognition/models/car_aerial_detection_yolo7_ITCVD_deepness.onnx'
session = ort.InferenceSession(onnx_model_path)
print("ONNX model loaded successfully!")

# Path to the large GeoTIFF map image
geo_tiff_path = '/content/drive/MyDrive/aerial_image_recognition/img/geotiff/srodmiescie.tiff'

# Load the GeoTIFF as a large image and extract the geotransform
dataset = gdal.Open(geo_tiff_path)
geotransform = dataset.GetGeoTransform()
img = dataset.ReadAsArray().transpose((1, 2, 0))  # Read GeoTIFF directly to preserve georeferencing
h, w = img.shape[:2]

# Ensure the image has 3 channels (RGB), if it has 4 (RGBA), remove the alpha channel
if img.shape[2] == 4:
    img = img[:, :, :3]

# Define window size and stride based on the larger scale
window_size = 1200  # Adjusted size for better coverage
stride = int(window_size * 0.91)  # 9% overlap between tiles

# Initialize an empty list to store car/truck centroids and confidences
car_truck_centroids = []

# Calculate the total number of tiles for the progress bar
total_tiles = ((h - window_size) // stride + 1) * ((w - window_size) // stride + 1)

# Function to convert pixel coordinates to geographic coordinates
def pixel_to_geo(pixel_x, pixel_y, geotransform):
    geo_x = geotransform[0] + pixel_x * geotransform[1] + pixel_y * geotransform[2]
    geo_y = geotransform[3] + pixel_x * geotransform[4] + pixel_y * geotransform[5]
    return geo_x, geo_y

# Function to run inference using ONNX model
def run_onnx_inference(window):
    # Preprocess the image (resize to the expected input size, i.e., 640x640 for ONNX model)
    original_size = window.shape[:2]
    input_image = cv2.resize(window, (640, 640))  # Resize to model input size
    input_image = np.transpose(input_image, (2, 0, 1))  # Change from HWC to CHW format
    input_image = input_image.astype(np.float32) / 255.0  # Normalize image
    input_image = np.expand_dims(input_image, axis=0)  # Add batch dimension

    # Run inference
    input_name = session.get_inputs()[0].name
    outputs = session.run(None, {input_name: input_image})

    return outputs, original_size

# Function to extract bounding boxes, centroids, and confidence scores from the ONNX model output
def process_onnx_output(outputs, original_size):
    bounding_boxes = outputs[0][0]  # Assuming the output is structured similarly to your previous example

    centroids = []
    for bbox in bounding_boxes:
        x1, y1, x2, y2, confidence, class_id = bbox
        if confidence > 0.5:  # Only consider high-confidence detections
            # Calculate the centroid in the resized window (640x640 space)
            centroid_x = (x1 + x2) / 2
            centroid_y = (y1 + y2) / 2

            # Scale the centroid back to the original window size
            scale_x = original_size[1] / 640.0
            scale_y = original_size[0] / 640.0
            centroid_x *= scale_x
            centroid_y *= scale_y

            centroids.append((centroid_x, centroid_y, class_id, confidence))

    return centroids

# Loop through the image using a sliding window approach with a progress bar
with tqdm(total=total_tiles, desc="Processing Entire Image") as pbar:
    for y in range(0, h, stride):
        for x in range(0, w, stride):
            # Extract the current window (ensure you don't go beyond image borders)
            window = img[y:min(y + window_size, h), x:min(x + window_size, w)]

            # Run inference using ONNX model
            outputs, original_size = run_onnx_inference(window)

            # Process the ONNX output and get centroids
            centroids = process_onnx_output(outputs, original_size)

            # Loop through the centroids and convert pixel coordinates to geographic coordinates
            for centroid_x, centroid_y, class_id, confidence in centroids:
                # Adjust the centroid based on the window's position in the full image
                centroid_x += x
                centroid_y += y

                # Convert pixel coordinates to geographic coordinates
                geo_x, geo_y = pixel_to_geo(centroid_x, centroid_y, geotransform)

                # Store the centroid, class (Car/Truck), and confidence
                car_truck_centroids.append({
                    'geometry': Point(geo_x, geo_y),
                    'class': 'Car' if class_id == 0 else 'Truck',
                    'confidence': confidence
                })

            # Update the progress bar
            pbar.update(1)

# Create a GeoDataFrame from the centroids list
car_truck_gdf = gpd.GeoDataFrame(car_truck_centroids)

# Set the CRS to match the GeoTIFF CRS
car_truck_gdf.set_crs(epsg=3857, inplace=True)  # Using EPSG:3857 for Web Mercator CRS

# Check if the output directory exists, if not, create it
output_dir = '/content/drive/MyDrive/aerial_image_recognition/gis/shp/'
os.makedirs(output_dir, exist_ok=True)

# Save the car and truck centroids, class, and confidence as a GeoJSON
output_geojson_path = os.path.join(output_dir, 'car_centroids_onnx.geojson')
car_truck_gdf.to_file(output_geojson_path, driver='GeoJSON')

print(f"Car and truck centroids with confidence saved as GeoJSON to: {output_geojson_path}")


ONNX model loaded successfully!


Processing Entire Image: 480it [01:02,  7.69it/s]


Car and truck centroids with confidence saved as GeoJSON to: /content/drive/MyDrive/aerial_image_recognition/gis/shp/car_centroids_onnx.geojson
